<a href="https://colab.research.google.com/github/Danielasilvac/Livro-POO-com-Python-Daniela-Silva-Cunha/blob/main/06_Apostila_13_Exce%C3%A7%C3%B5es_e_Erros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercícios - Exceções e Erros

In [ ]:
import abc

class SaldoInsuficienteError(RuntimeError): # Exceção personalizada para saldo insuficiente
    pass

class Tributavel(abc.ABC): # Interface abstrata que define o contrato para classes tributáveis
    @abc.abstractmethod
    def get_valor_imposto(self):
        pass

class Conta(abc.ABC): # Classe base abstrata para contas bancárias
    def __init__(self, titular, numero, saldo=0.0, limite=1000.0):
        self._titular = titular
        self._numero = numero
        self._saldo = saldo
        self._limite = limite

    @property
    def saldo(self):
        return self._saldo

    def depositar(self, valor): # Método para depositar valores positivos
        if valor < 0:
            raise ValueError('Você tentou depositar um valor negativo.')
        self._saldo += valor

    def sacar(self, valor):  # Método para sacar valores, com verificação de saldo
        if valor < 0:
            raise ValueError('Você tentou sacar um valor negativo.')
        if self._saldo < valor:
            raise SaldoInsuficienteError('Saldo insuficiente.')
        self._saldo -= valor  # Sem taxa, pois taxa só em ContaCorrente

    def __repr__(self):
        return f"{self.__class__.__name__}('{self._titular}', '{self._numero}')"

    def __str__(self):
        return f"{self.__class__.__name__} - Número: {self._numero}, Titular: {self._titular}, Saldo: R$ {self._saldo:.2f}"

class ContaCorrente(Conta): # ContaCorrente com taxa de saque e cálculo de imposto
    def get_valor_imposto(self):
        return self._saldo * 0.01 # Imposto de 1% sobre o saldo

    def sacar(self, valor): # Método de saque com cobrança de taxa fixa de R$ 0,10
        if valor < 0:
            raise ValueError('Você tentou sacar um valor negativo.')
        if self._saldo < valor + 0.10:
            raise SaldoInsuficienteError('Saldo insuficiente.')
        self._saldo -= (valor + 0.10)

class ContaPoupanca(Conta): # ContaPoupanca não é tributável, sem impostos nem taxas extras
    def depositar(self, valor):
        if valor < 0:
            raise ValueError('Você tentou depositar um valor negativo.')
        self._saldo += valor

class ContaInvestimento(Conta): # ContaInvestimento com maior rendimento e imposto mais alto
    def atualizar(self, taxa):
        self._saldo += self._saldo * taxa * 5 # Multiplicador de rendimento

    def get_valor_imposto(self):
        return self._saldo * 0.03 # Imposto de 3% sobre o saldo

class SeguroDeVida: # Classe SeguroDeVida, tributável com valor fixo + percentual
    def __init__(self, valor, titular, numero_apolice):
        self._valor = valor
        self._titular = titular
        self._numero_apolice = numero_apolice

    def get_valor_imposto(self): # Imposto fixo de R$ 50 + 5% do valor
        return 50 + self._valor * 0.05

    def __repr__(self):
        return f"SeguroDeVida('{self._titular}', '{self._numero_apolice}')"

class ManipuladorDeTributaveis: # Classe para calcular impostos de objetos tributáveis
    def calcular_impostos(self, lista_tributaveis):
        total = 0
        for item in lista_tributaveis:
            if isinstance(item, Tributavel): # Verifica se implementa Tributavel
                total += item.get_valor_imposto()
            else:
                print(item.__repr__(), "não é um tributável.")
        return total

class CaixaEletronico: # CaixaEletronico simula operações de saque e depósito com tratamento de erros
    def sacar(self, conta, valor):
        try:
            conta.sacar(valor)
            print(f"Saque de R$ {valor:.2f} realizado com sucesso.")
        except ValueError as e:
            print(f"Erro: {e}")
        except SaldoInsuficienteError as e:
            print(f"Erro: {e}")

    def depositar(self, conta, valor):
        try:
            conta.depositar(valor)
            print(f"Depósito de R$ {valor:.2f} realizado com sucesso.")
        except ValueError as e:
            print(f"Erro: {e}")

# Registrar classes como tributáveis
Tributavel.register(ContaCorrente)
Tributavel.register(ContaInvestimento)
Tributavel.register(SeguroDeVida)

# Exemplo de uso
if __name__ == '__main__':
   # Criação das contas e seguro
    cc = ContaCorrente('Vanessa', '123-4', 1000.0)
    cp = ContaPoupanca('Miguel', '123-5', 500.0)
    ci = ContaInvestimento('Henrique', '123-6', 300.0)
    seguro = SeguroDeVida(200.0, 'Adriana', '345-77')

    caixa = CaixaEletronico()

    # Testes de saque e depósito
    caixa.sacar(cc, -100) # Deve lançar erro de valor negativo
    caixa.sacar(cc, 2000) # Deve lançar erro de saldo insuficiente
    caixa.sacar(cc, 100) # Saque válido com taxa de R$ 0,10

    caixa.depositar(cc, -50) # Deve lançar erro de valor negativo
    caixa.depositar(cc, 200) # Depósito válido

 # Exibe o estado atual da conta corrente após as operações
    print("\nResumo da conta após operações:")
    print(cc)

    # Teste de tributação
    lista = [cc, ci, seguro, cp] # cp (ContaPoupanca) não é tributável
    manipulador = ManipuladorDeTributaveis()
    total_impostos = manipulador.calcular_impostos(lista)

 # Exibe o valor total de impostos
    print(f"\nTotal de impostos calculados: R$ {total_impostos:.2f}")

Erro: Você tentou sacar um valor negativo.
Erro: Saldo insuficiente.
Saque de R$ 100.00 realizado com sucesso.
Erro: Você tentou depositar um valor negativo.
Depósito de R$ 200.00 realizado com sucesso.

Resumo da conta após operações:
ContaCorrente - Número: 123-4, Titular: Vanessa, Saldo: R$ 1099.90
ContaPoupanca('Miguel', '123-5') não é um tributável.

Total de impostos calculados: R$ 80.00
